In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import os
import requests

import util as u

from bs4 import BeautifulSoup
from glob import glob
from tqdm import tqdm_notebook as tqdm

pd.set_option('display.max_columns', 200)


In [7]:
#巡回先を作る
#枠とレースIDがあればマージできる？
result_infofile_list = glob("test/*/info_html/*") # 後々フォルダーが変わる
main_info_file = [x.replace("\\", "/") for x in result_infofile_list]

result_resultfile_list = glob("test/*/result_html/*") # 後々フォルダーが変わる
main_result_file = [x.replace("\\", "/") for x in result_resultfile_list]

target_info_list = [] #配列を空にする
target_result_list = [] #配列を空にする

match_up_info_list = [] #配列を空にする
match_up_result_list = [] #配列を空にする

#is_file = os.path.isfile("match_up.txt")

#if is_file:
#    with open("match_up.txt", "r") as tf:
#        match_up_list = tf.read().split(',')[:-1]
#    #途中から始めるためのコード書く
#    target_txts_list = set(result_file) - set(match_up_list)
#else:
#    target_txts_list = result_file

target_info_list = main_info_file[9:10]
target_result_list = main_result_file[0:1]

In [8]:
#データフレームを4個作る
main_info = pd.DataFrame(index=[])
main_info_dummy = pd.DataFrame(index=[])
main_result = pd.DataFrame(index=[])
main_result_dummy = pd.DataFrame(index=[])

New_col = 'insert_1_1 insert_1_2 insert_2_1 insert_2_2 insert_3_1 insert_3_2 insert_4_1 insert_4_2 insert_5_1 insert_5_2 insert_6_1 insert_6_2 insert_7_1 insert_7_2 \
            start_timing_1_1 start_timing_1_2 start_timing_2_1 start_timing_2_2 start_timing_3_1 start_timing_3_2 start_timing_4_1 start_timing_4_2 start_timing_5_1 start_timing_5_2 start_timing_6_1 start_timing_6_2 start_timing_7_1 start_timing_7_2 \
            rank_1_1 rank_1_2 rank_2_1 rank_2_2 rank_3_1 rank_3_2 rank_4_1 rank_4_2 rank_5_1 rank_5_2 rank_6_1 rank_6_2 rank_7_1 rank_7_2'.split()

dfs_all = pd.DataFrame(index=[])
dfs_dummy = pd.DataFrame(index=[])

In [55]:
###後で消す↓###
main_info = pd.DataFrame(index=[])
main_info_dummy = pd.DataFrame(index=[])
###↑後で消す###

# info_htmlの加工をする
# race_id、枠、選手ID、級別、体重(直前情報から取る)、調整重量、F数、L数、平均ST、モーター2連、モーター3連、初日進入、
# 初日ST、初日成績、～ 、最終日、展示タイム、チルト、プロペラ、部品交換

for target_info in tqdm(target_info_list):
    for i in range(1, 7):
        target_info_day = target_info[-19:-5]
        Pit_No = f"{i:02}"
        
        #レース結果を保存したhtmlファイルとそのDF
        target_info_day = target_info[-19:-5]
        dfs = pd.read_html(target_info, encoding="utf-8")
        race_info1_df = dfs[0]
        
        #dfs[0]の整理
        df_player_info = u.split_inner_data(u.parse(race_info1_df), target_info_day)
        df_player_info = df_player_info.iloc[i-1]

        player_ID = df_player_info["pid"]

        #複勝率、三連率を計算
        player_win_rate = u.player_win_rate_cal(target_info_day, player_ID)
        se_player_win_rate = player_win_rate.iloc[0]

        #df_player_info と player_win_rateをマージする(ridで)
        df_open_grades = u.Open_Grades_sort(target_info).iloc[i-1]

        
        #main_info_dummy = pd.concat([df_player_info, player_win_rate])
        #main_info_dummy = pd.concat([df_player_info, df_open_grades])
        main_info_dummy = pd.merge(df_player_info, se_player_win_rate, on="rid", how="inner")
        #main_info_dummy = df_player_info.append(df_open_grades)
        #main_info_dummy = main_info_dummy.transpose()
        
        #display(main_info_dummy)
        
        main_info = main_info.append(main_info_dummy, ignore_index=True)

#print(df_player_info[0] == player_win_rate.iloc[0, 0])

#display(df_player_info)
#display(main_info)
display(player_win_rate)
#display(dfs)
#display(dfs[1])
#display(dfs[2])


C:\Users\rstta\AppData\Local\Temp/ipykernel_11016/4076057518.py:10: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for target_info in tqdm(target_info_list):


  0%|          | 0/1 [00:00<?, ?it/s]

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [ ]:
#      a    b
# 0  a_1  b_1
# 1  a_2  b_2
# 2  a_3  b_3

#      a    c
# 0  a_1  c_1
# 1  a_2  c_2
# 2  a_4  c_4

In [138]:
dfs_dummy = pd.DataFrame(index=[])
dfs = pd.read_html("row_html/01/info_html/01_20180520_10.html", encoding="utf-8")
dfs_ = dfs[0]["レースNo（艇番色）進入コースSTタイミング成績", "レースNo（艇番色）進入コースSTタイミング成績"]

for i in range(1, 25, 4):
    x = pd.concat([dfs_.iloc[i], dfs_.iloc[i+1], dfs_.iloc[i+2]])
    x_ = pd.DataFrame(x)
    x_ = x_.reset_index()
    #x_ = x_.drop('index', axis=1, inplace=True)
    x_ = x_.transpose()
    x_ = x_.iloc[1]
    #x_ = x_.insert(0, "race_id", "01_20180520_10")
    dfs_dummy = dfs_dummy.append(x_)
    
dfs_dummy.columns = New_col

dfs_dummy.reset_index(drop=True, inplace=True)
#dfs_all = dfs_dummy.insert(0, 'new_column', 'value')

display(dfs_dummy)

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2901: PerformanceWarning: indexing past lexsort depth may impact performance.
  result = self._run_cell(


,insert_1_1,insert_1_2,insert_2_1,insert_2_2,insert_3_1,insert_3_2,insert_4_1,insert_4_2,insert_5_1,insert_5_2,insert_6_1,insert_6_2,insert_7_1,insert_7_2,start_timing_1_1,start_timing_1_2,start_timing_2_1,start_timing_2_2,start_timing_3_1,start_timing_3_2,start_timing_4_1,start_timing_4_2,start_timing_5_1,start_timing_5_2,start_timing_6_1,start_timing_6_2,start_timing_7_1,start_timing_7_2,rank_1_1,rank_1_2,rank_2_1,rank_2_2,rank_3_1,rank_3_2,rank_4_1,rank_4_2,rank_5_1,rank_5_2,rank_6_1,rank_6_2,rank_7_1,rank_7_2
0,2,5,5,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.15,.11,.18,NaN,NaN,NaN,.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,４,４,３,NaN,NaN,NaN,２,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,3,1,NaN,4,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.07,.24,.31,NaN,.20,NaN,.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,２,３,３,NaN,１,NaN,２,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,2,1,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.20,.22,.16,.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,３,３,１,６,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,1,3,NaN,NaN,NaN,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.13,.22,.24,NaN,NaN,NaN,.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,１,１,４,NaN,NaN,NaN,３,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,4,5,NaN,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.08,.18,.35,NaN,NaN,NaN,.29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,３,５,６,NaN,NaN,NaN,６,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4,NaN,1,3,NaN,NaN,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,.19,NaN,.25,.12,NaN,NaN,.22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,２,NaN,３,５,NaN,NaN,５,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# result_htmlの加工をする

In [ ]:
url = 'https://www.boatrace.jp/owpc/pc/race/racelist?rno=2&jcd=02&hd=20220413'
race_id = u.url2raceid(url)
u.split_inner_data(race_info1_df, race_id)

print(u.url2raceid(url))

In [101]:
race_id = "11_20140102_10"
day = race_id[3:7] + "-" + race_id[7:9] + "-" + race_id[9:11]
open_place = int(race_id[0:2])
print(open_place)

11


### データが取れてないところからデータを取る処理を書く

### 現在開催されているデータを引っ張ってくる

### PyCaretってのを使ってみるよ

アパートが1500万円でした
月収入が15万
銀行から借金が1500万円
借金返済が15万
残金 650万くらい
15年でアパートが手に入るよ手出しほぼなく
